In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
#import tensorflow as tf
#from tensorflow.python.framework import ops
import pandas as pd
%matplotlib inline
np.random.seed(1)

In [ ]:
#import lasagne 
#import theano.tensor as T
import pandas as pd
import numpy as np
import os
import sys
#import _pickle 

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials= GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)
#4/_QD_IUfoboV1WT_E8cG35ucGqNxRC0m0s7ZSWAW5iP6AVOWRpxwrWp0

In [ ]:
link='https://drive.google.com/open?id=18ayHUA1M7tcz-N6HZRvBnZkMCbFedWHv'
#link2='https://drive.google.com/open?id=1Tj4fEXcoWBFr2ixSTd7cb5_2lPrm4F9a'
poof,file_id=link.split("=")
#file_id2='1Tj4fEXcoWBFr2ixSTd7cb5_2lPrm4F9a'
downloaded=  drive.CreateFile({'id':file_id})
#downloaded2= drive.CreateFIled2':file_id2})
downloaded.GetContentFile('compound_features_256.csv') #Training dataset
#downloaded2.GetContentFile('AID373AID439red_test.csv') #Testing dataset
data= pd.read_csv('compound_features_256.csv',sep="\t",header=0)
#data2= pd.read.csv('AID373AID439red_test.csv')


In [ ]:
def GetFile(path,file_name,delim="\t",file_id=None):
  
  if not file_id:
    poof,file_id=path.split("=")
  
  downloaded=  drive.CreateFile({'id':file_id})
  downloaded.GetContentFile(file_name) #Training dataset
  data= pd.read_csv(file_name, sep=delim, header=0)
  
  return data

In [ ]:
file_id2='1qJC5BAbBUXv4VHI0eIg92G-n9z8XF-Fy'
xyz=drive.CreateFile({'id':file_id2})
xyz.GetContentFile('compound_additional_physchem_features.txt')
data2 = pd.read_csv('compound_additional_physchem_features.txt',sep="\t",header=0)

In [ ]:
file_id4='1R9NC1bIu7pJfrMVIfkNelGy1ijPaJKR3'
abcd=drive.CreateFile({'id':file_id4})
abcd.GetContentFile('Protein_Descriptors_20_sections.txt')
prot_desc = pd.read_csv('Protein_Descriptors_20_sections.txt',sep="\t",header=0)

In [ ]:
file_id3='1SGX3gptAPDm_ZGo0_fQHdDV8ymUMwSvR'
abc=drive.CreateFile({'id':file_id3})
abc.GetContentFile('activity_train.txt')
trainset = pd.read_csv('activity_train.txt',sep=" ",header=0)
trainset['Target_ID'] = 'CHEMBL' + trainset['Target_ID'].astype(str)

In [ ]:
prot=pd.DataFrame(trainset['Target_ID'])
prot=prot.drop_duplicates(subset=None, keep='first', inplace=False)
prot2=prot_desc.set_index('TGT_CHEMBL_ID').join(prot.set_index('Target_ID'))
mergedStuff = pd.merge(prot_desc,prot , left_on=['TGT_CHEMBL_ID'],right_on=['Target_ID'], how='inner')
mergedStuff.shape

In [ ]:
totalcmp = data.set_index('Compound_ID').join(data2.set_index('CMP_CHEMBL_ID'))
totalcmp.insert(0,"Compound_ID",list(totalcmp.index),True)
totalcmp=totalcmp.drop_duplicates(subset=None, keep='first', inplace=False)
train_val = pd.DataFrame(trainset[['Compound_ID','Active']])
targetprots = trainset.set_index('Target_ID').join(prot_desc.set_index('TGT_CHEMBL_ID'))
#targetprots = targetprots.drop(columns=['Active'])
targetprots.insert(0,"Target_ID",list(targetprots.index),True)
targetprots['Compound_ID'] = 'CHEMBL' + targetprots['Compound_ID'].astype(str)
X_pcm = targetprots.set_index('Compound_ID').join(totalcmp.set_index("Compound_ID"))
#X_pcm


In [ ]:
y=trainset.copy()
prot

In [ ]:
#https://drive.google.com/open?id=1hJ28sWQ8i0oim5kCa3Fm219RvGFN2YuD
file_id5='1hJ28sWQ8i0oim5kCa3Fm219RvGFN2YuD'
abc=drive.CreateFile({'id':file_id5})
abc.GetContentFile('activity_test_eval.txt')
testset = pd.read_csv('activity_test_eval.txt',sep=" ",header=0)
testset['Target_ID'] = 'CHEMBL' + testset['Target_ID'].astype(str)
testset['Compound_ID'] = 'CHEMBL' + testset['Compound_ID'].astype(str)

In [ ]:
targetprots_test = testset.set_index('Target_ID').join(prot_desc.set_index('TGT_CHEMBL_ID'))
targetprots_test.insert(0,"Target_ID",list(targetprots_test.index),True)
#targetprots_test['Compound_ID'] = 'CHEMBL' + targetprots_test['Compound_ID'].astype(str)
X_pcm_Test = targetprots_test.set_index('Compound_ID').join(totalcmp.set_index("Compound_ID"))
targetprots.shape

In [ ]:
for item in prot:
  for it in trainset:
    if item 

In [ ]:
def scale_data(X_train, X_test):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    # Fit the scaler based on the training data, then apply the same scaling to both training and test sets.
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    

#    if not train_fp.empty or test_fp.empty:
#        train_fp = pd.DataFrame(train_fp)
#        test_fp = pd.DataFrame(test_fp)
#        X_train = pd.concat([X_train, train_fp], axis = 1)
#        X_test = pd.concat([X_test, test_fp], axis = 1)


    X_train = np.array(X_train, dtype=np.float32)
    X_test = np.array(X_test, dtype=np.float32)

    return X_train, X_test


In [ ]:
#y_train=[[]]
#y_train.shape[0] = X_pcm.shape[0]
#y_train.shape[1] = prot_desc.shape[0]

train_x, test_x = scale_data(train_x, test_x)



In [ ]:
test_y = pd.get_dummies(test_y)


In [ ]:
test_y.sum()
#test_y['CHEMBL261'].value_counts()

In [ ]:
#len(test_y[test_y['CHEMBL6165']==1].index.tolist())
X_pcm_test[X_pcm_Test['Target_ID']='CHEMBL261'].tolist().count()

In [ ]:
import io

# data = pd.read_csv(io.BytesIO(uploaded['neural.csv']))
#data = pd.read_csv(content/neural4.csv)
data_tr = X_pcm.copy()
data_test = X_pcm_Test.copy()


train_x = data_tr.iloc[:,2:].values
test_x = data_test.iloc[:,2:].values

#train_x, test_x = scale_data(train_x, test_x)
test_y = data_test.iloc[:,0]

train_y = data_tr.iloc[:,0].values

for idx,item in enumerate(data_test):
  if data_test.iloc[idx,1] < 6.5:
    test_y.iloc[idx] = None
  
    
#reg_train_y=data_tr['Active']
label=[]
    
#for ind,i in enumerate(data_tr['Active']):
  
#  if i>=6.5:
#    x="Active"

#  else:
#    x="Inactive"
    
#  label.append(x)
  
''' 
for ind,trgt in enumerate(data_test['Target_ID']):
  for idx,target in enumerate(prot_desc['TGT_CHEMBL_ID']):
    if target==trgt:
      y[idx].append(1)
    else:
      y[ind].append(0)
      
label2=[]

for ind,i in enumerate(data_test['Active']):
  
  if i>=6.5:
    x="Active"

  else:
    x="Inactive"
    
  label2.append(x)



train_x = train_x.astype('float32')
train_y = pd.get_dummies(train_y)

test_x = test_x.astype('float32')
test_y = pd.get_dummies(test_y)
'''

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from keras import losses

rate = 0.25

clf = Sequential()
clf.add(Dense(4000, input_dim = 545, activation = 'relu'))
#clf.add(Dropout(rate))
clf.add(Dense(2000, input_dim = 4000, activation = 'relu'))
#clf.add(Dropout(rate))
clf.add(Dense(1000, input_dim = 2000, activation = 'relu'))
#clf.add(Dropout(rate))
clf.add(Dense(1224, input_dim = 1000, activation = 'softmax'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.8, nesterov=True)
adm = optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
clf.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'] )
 # metrics=['accuracy'], loss='mean_squared_error'

In [ ]:
clf.fit(test_x, test_y , batch_size = 100, epochs = 50, validation_split=0.2) 

In [ ]:
test_x.shape

In [ ]:
from google.colab import files
files.download('neural.csv')